In [1]:
using Pkg

Pkg.activate("..")
using Revise
using PeriDyn
using PDMesh

  Activating environment at `/mnt/D/git_repos/PeriDyn/Project.toml`
┌ Info: Precompiling PeriDyn [c3db0ce0-6d7e-41b5-be11-bcb6aafada44]
└ @ Base loading.jl:1342
  ** incremental compilation may be fatally broken for this module **



In [ ]:
obj = Cuboid([0 6; 0 1; 0 1])
out = create(obj, resolution=0.1,rand_=0.05, type=1)

Es = 100 # GPa 
nu = 0.3333         # Poisson's Ratio 
K = Es/3/(1-2nu)    # Bulk Modulus
G = Es/2/(1+nu)     # Shear Modulus
den = 1000.0        # Density in Kg m ^-3
horizon = 0.3
σ = 0.7Es
println(σ)
s = 100.0

mat_gen = GeneralMaterial(out[:y], out[:v], out[:x], out[:volume], out[:type], 0.3; max_neigh=200)
# mat_spec = ElastoPlasticSolidSpecific([K] ,[G] ,[s] ,[den], [σ])
mat_spec = OrdinaryStateBasedSpecific([K] ,[G] ,[s] ,[den])
block = PeridynamicsMaterial(mat_gen, mat_spec)



BC1 = MoveBC(out[:y][1, :] .< 0.5, [-0.05, 0.0, 0.0])#, 40; applyafter=60)
BC2 = MoveBC(out[:y][1, :] .> 5.5, [ 0.05, 0.0, 0.0])#, 40; applyafter=60)

k = 0.000005
exp=4.0
RM1 = NonLinearRepulsionModel(exp,k, block; distanceX=3, max_neighs=200)

dt = 0.5
env = PeriDyn.Env(1, [block], [RM1], [BC1, BC2], dt)

udf = Int(1/dt)


In [ ]:
udf = 1
Steps = 500*udf

PeriDyn.simulate([env], Steps; solver=:vv, filewrite_freq=udf, neigh_update_freq=udf, file_prefix="Tensile1",start_at=0)